In [3]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
# dir_path = './' #local
dir_path = './drive/MyDrive/BAKA/' #colab

In [4]:
#uncomment in colab
from google.colab import drive
drive.mount('/content/drive')
# pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', sep=' ', index_col=False )
# test = np.genfromtxt(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', delimiter=' ',dtype='float64')
df = pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', header=None, sep=' ')
df = df.drop(columns=[31])
sp_df = np.array_split(df, 2)
sp_df[0]

Mounted at /content/drive


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,152.028,174.270,114.872,135.012,115.821,169.357,108.251,86.5737,86.4014,60.6628,7.49661,368.805,-11.39230,35.47780,372.833,-21.5318,39.6854,348.592,7.46019,38.7144,326.577,19.15310,37.6444,296.277,5.277580,5.69833,303.563,-4.53744,7.62196,12.33870,35.1608
1,106.969,174.270,138.093,122.560,112.017,151.941,139.409,67.5126,93.4567,141.7770,25.46860,371.098,-13.88750,43.57630,381.204,-39.2998,39.0602,356.814,-7.43494,49.2235,343.591,0.97681,54.9755,295.547,-61.058000,8.98806,317.394,-4.13179,85.07790,5.64757,43.2827
2,115.931,174.270,114.917,134.286,175.119,127.521,177.996,58.0159,69.8662,178.0840,49.53920,383.612,-5.43277,56.81450,405.251,-43.7347,35.5285,371.161,-20.33810,34.9854,355.458,-2.39748,82.3194,336.737,-44.982000,13.94950,337.090,-7.44972,63.64330,21.32290,29.9727
3,129.774,90.000,151.379,164.207,147.675,133.193,153.299,53.3455,64.2682,55.6463,68.55800,397.308,-25.93310,57.00650,390.412,-74.1331,42.2847,387.940,-17.66460,43.7154,368.581,6.15326,45.3346,348.796,1.379250,19.70390,370.834,-9.76106,19.74900,17.69820,71.4496
4,168.765,174.270,156.177,141.628,146.640,128.245,177.981,80.4555,66.8833,71.7721,72.51100,366.007,-19.21010,35.86220,344.209,-96.2058,65.4289,368.487,-5.98384,51.3009,373.351,7.91760,45.8886,362.047,13.636300,27.27890,388.642,-8.96836,15.10440,19.16930,97.3733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,155.025,173.721,137.026,141.175,142.413,144.205,171.451,71.6357,76.7435,77.4298,32.57980,385.636,-55.35270,3.71505,339.714,-94.7966,50.2944,397.548,-18.37210,54.0502,394.459,-5.88642,50.2178,384.405,1.723980,18.24680,398.022,-10.44550,15.77150,9.59649,110.1400
96,156.901,173.395,136.497,140.909,141.700,144.869,170.816,71.1895,76.4634,77.2505,31.79950,386.771,-56.38500,4.50401,341.073,-95.1066,49.8712,398.885,-19.14140,54.1542,395.603,-6.63851,50.7962,385.537,0.938872,18.37180,398.392,-10.57410,15.82180,9.31662,109.5280
97,158.522,173.625,136.929,141.031,141.552,145.678,170.873,71.2914,76.3473,76.4080,31.83980,387.572,-57.14600,4.90973,341.923,-95.7195,49.8744,399.897,-19.53570,54.2922,396.469,-7.09323,50.9274,386.211,0.622400,18.48320,398.925,-10.66650,15.72430,9.22443,109.0020
98,158.859,174.598,137.177,140.760,141.265,147.142,171.260,71.2260,76.3613,76.1615,32.39350,388.048,-57.93710,4.57446,342.496,-95.8547,49.8242,400.669,-19.98300,54.4927,397.135,-7.47433,51.2412,386.845,0.361023,18.55280,399.310,-10.78070,15.59450,8.98942,109.1230


In [5]:
files = os.listdir(dir_path + 'DataCollection')
files

['4', '2', '6', '1', '3', '5']

In [6]:
x_train = [];
y_train = [];

x_test = [];
y_test = [];
for i in files:
    samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
    num_tests = int(len(samples)/5);
    shuffle(samples, random_state = 0)
    for k in range(0, num_tests):
#         df = np.genfromtxt(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], delimiter=' ',dtype='float64')
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(columns=[31])
        sp_df = np.array_split(df, 2)
        for p in sp_df:
            x_test.append(p.to_numpy())
            y_test.append(int(i)-1);
    
    for k in range(num_tests, len(samples)):
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(columns=[31])
        sp_df = np.array_split(df, 2)
        for p in sp_df:
            x_train.append(p.to_numpy())
            y_train.append(int(i)-1);
    
    print(len(samples), ' ', num_tests)
    
x_train = np.array(x_train)
y_train = np.array(y_train);

x_test = np.array(x_test)
y_test = np.array(y_test)

205   41
256   51
151   30
301   60
151   30
211   42


In [7]:
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)

print(x_train)

x_train.shape:  (2042, 100, 31)
y_train.shiape:  (2042,)
x_test.shape:  (508, 100, 31)
y_test.shape:  (508,)
[[[140.263   155.047   153.056   ...  14.6581   10.9596   14.1146 ]
  [139.707   154.679   150.799   ...  14.6448   11.2233   14.2539 ]
  [140.28    153.484   149.955   ...  14.4216   10.9631   14.1897 ]
  ...
  [147.243   146.769   148.691   ...  24.7754   12.0004   19.3087 ]
  [146.985   146.403   148.536   ...  25.2561   11.4941   19.4561 ]
  [147.045   147.45    149.014   ...  23.664    11.4556   20.0084 ]]

 [[146.053   146.94    146.925   ...  22.0255   10.9032   21.6817 ]
  [145.846   148.58    148.411   ...  20.4109   10.9894   22.0994 ]
  [145.929   149.622   150.173   ...  20.5501   10.91     21.5744 ]
  ...
  [146.804   158.72    155.316   ...  21.6282   10.9693   17.1487 ]
  [146.373   159.741   156.251   ...  22.2424   11.1586   17.4078 ]
  [145.241   159.593   156.114   ...  23.4668   11.1882   17.5774 ]]

 [[143.97    157.465   154.8     ...  25.9021   11.1725   1

In [8]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

In [9]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)

num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, newshape=(-1, num_features))
x_test = min_max_scaler.transform(x_test)
x_test = np.reshape(x_test, newshape=(num_instances, num_time_steps, num_features))

x_test, y_test = shuffle(x_test, y_test, random_state=0)


In [10]:
# x_train = np.delete(x_train, [0,1,2,3,4,5,6,7,8], 0)
# y_train = np.delete(y_train, [0,1,2,3,4,5,6,7,8], 0)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)
# print(y_train)
# print("\n")
# print(y_test)

# y_train = y_train.astype('int')

# print(y_train)


x_train.shape:  (2042, 100, 31)
y_train.shiape:  (2042,)
x_test.shape:  (508, 100, 31)
y_test.shape:  (508,)


In [11]:
model = Sequential()
model.add(LSTM(units=100, input_shape=x_train.shape[1:], return_sequences=True,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# model.add(Dense(30, activation='softmax'))

model.add(Dense(6, activation='softmax',dtype='float64'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 100)          52800     
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 100)          400       
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 100)          80400     
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100)          400       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 100)          8

In [12]:
opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-5)

checkpoint_filepath = 'Checkpoints'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)


gestures = model.fit(x = x_train,
            y = y_train,
            epochs=200,
            validation_split=0.1, #split 10% of the trainning set for the validation set,
            batch_size=80,
            callbacks=[model_checkpoint_callback],
            shuffle=True
         )

Epoch 1/200
23/23 [==============================] - 42s 257ms/step - loss: 1.6793 - accuracy: 0.4241 - val_loss: 1.7454 - val_accuracy: 0.2146
Epoch 2/200
23/23 [==============================] - 5s 203ms/step - loss: 0.3826 - accuracy: 0.8845 - val_loss: 1.6815 - val_accuracy: 0.3171
Epoch 3/200
23/23 [==============================] - 5s 206ms/step - loss: 0.2610 - accuracy: 0.9316 - val_loss: 1.5989 - val_accuracy: 0.4244
Epoch 4/200
23/23 [==============================] - 5s 205ms/step - loss: 0.2113 - accuracy: 0.9483 - val_loss: 1.4961 - val_accuracy: 0.4780
Epoch 5/200
23/23 [==============================] - 5s 204ms/step - loss: 0.1627 - accuracy: 0.9574 - val_loss: 1.3390 - val_accuracy: 0.6098
Epoch 6/200
23/23 [==============================] - 5s 206ms/step - loss: 0.1468 - accuracy: 0.9604 - val_loss: 1.1434 - val_accuracy: 0.7317
Epoch 7/200
23/23 [==============================] - 5s 205ms/step - loss: 0.1377 - accuracy: 0.9669 - val_loss: 0.9546 - val_accuracy: 0.800

In [13]:
# model.save(dir_path + 'Models/gestures.h5')

In [14]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=80)
print("test loss, test acc:", results)
model.save(dir_path + 'Models/gestures_100', save_format='tf')

Evaluate on test data
7/7 [==============================] - 1s 74ms/step - loss: 0.1182 - accuracy: 0.9724
test loss, test acc: [0.11823232471942902, 0.9724409580230713]


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_100/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_100/assets


In [15]:
# new_model = load_model(dir_path + 'Models/gestures.h5')
# print("Evaluate on test data")
# results = new_model.evaluate(x_test, y_test, batch_size=80)
# print("test loss, test acc:", results)

In [16]:
min_max_scaler.data_min_

array([ 9.30245e+01,  8.99999e+01,  9.00456e+01,  9.00455e+01,
        9.00455e+01,  6.54812e+01,  2.57840e+01,  2.77705e+01,
        2.79286e+01,  2.04904e+01, -4.24721e+02,  6.97324e+01,
       -2.97270e+02, -4.20951e+02,  5.95510e+01, -3.15328e+02,
       -4.48325e+02,  6.33254e+01, -3.17749e+02, -4.46258e+02,
        8.67906e+01, -3.13954e+02, -4.57886e+02,  8.58241e+01,
       -3.02162e+02, -4.42757e+02,  1.03148e+02, -2.82183e+02,
        1.86895e-01,  6.48663e-02,  1.07620e-01])